# GEOPHYS 257 (Winter 2023)
[//]: <> (Notebook Author: Thomas Cullison, Stanford University, Jan. 2023)

## Multicore Parallelization of Matrix Multiplication Using Numba

In this lab we will be using Numba to accelerate matrix-matrix multiplications by exploiting parallelism. Even most laptops today have Multicore CPUs, where a *core* is a microprocessor, and each core is usually a copy of the each other core. Accelerating the marrix-matrix multiplication operation is a good analog to accelerating other types of operators and computationally intense kernels, codes, and algorithms. Furthermore, the structure of matricies makes matrix-matrix multiplication a good place start learning how to parallelize code.


## External Resources
If you have any question regarding some specific Python functionality you can consult the official [Python documenation](http://docs.python.org/3/).

* [Numba](https://numba.readthedocs.io/en/stable/index.html): Documentation
* [Numba in 30 min](https://youtu.be/DPyPmoeUdcE): Conference presentation video


## Required Preperation
Please watch the following videos before starting the lab (each is pretty short):
* [Introduction to Parallel Computing](https://youtu.be/RNVIcm8-6RE)
* [Amdahl's Law](https://youtu.be/Axx2xuB-Xuo)
* [CPU Caching](https://youtu.be/KmairurdiaY)
* [Pipelining](https://youtu.be/zPmfprtdzCE)
* [Instruction Level Parallelism](https://youtu.be/ZoDUI3gTkDI)
* [Introduction to SIMD](https://youtu.be/o_n4AKwdfiA)


### Exercise 0

Please run the following cells.  Examine the result of the example function that makes use of the *timer* wrapper. Also please use this timer wrapper (defined below) for all the exercises that follow.

#### Load python modules (Note: you may need to install some Python packages for the modules below, e.g. Numba)

In [1]:
import numba
import numpy as np


from functools import wraps
from time import time, sleep
from itertools import permutations

#### Define a function-decorator for timing the runtime of your functions

Below is some code that you can use to wrap your functions so that you can time them individually.  The function defined imeadiately below the *timer()* is an example of how to use the warpper. In the cell that follows, execute this example function. (Note, for this timer, were are making use of something called *decorators*, but a discussion about this feature is outside the scope of this class.)


In [2]:
# defining a function decorator for timing other functions
def mytimer(func):
    @wraps(func)
    def wrap(*args, **kwargs):
        t_start = time() # Students look here
        result = func(*args, **kwargs)
        t_end = time() # Students also look here. This is how you can time things inside functions/classes
        print(f'Function: \'{func.__name__}({kwargs})\' executed in {(t_end-t_start):4.3f}s\n')
        return result
    return wrap
    

# Example of how to use. NOTE the "@mytimer" stated just above the function definition
@mytimer
def example_sum_timer_wrap(N):
    """ Sum the squares of the intergers, i, contained in [1-N] """
    return np.sum(np.arange(1,N+1)**2)

#### Run the example function

Run the example function for each of the following instances of $N = 10^5, 10^6, 10^7, 10^8$. Please examine the results, in particular, how the runtime changes with respect to $N$.


**Answer** the following questions in the markdown cell that follow the code. 
* For each factor-of-ten increase in $N$, roughly how much longer was the runtime of the function?
* Does this slowdown in the runtime make sense? Why or why not?

In [3]:
# Call the example function above for each value of N (try making one-call first, then loop)
N=[1e5,1e6,1e7,1e8]
print(len(N))

4


In [4]:
for i in range(len(N)):
    print(mytimer(example_sum_timer_wrap(N[i])))

Function: 'example_sum_timer_wrap({})' executed in 0.001s

<function mytimer.<locals>.wrap at 0x7f8b3eb4aa70>
Function: 'example_sum_timer_wrap({})' executed in 0.006s

<function mytimer.<locals>.wrap at 0x7f8b3ebba680>
Function: 'example_sum_timer_wrap({})' executed in 0.055s

<function mytimer.<locals>.wrap at 0x7f8b3ebba680>
Function: 'example_sum_timer_wrap({})' executed in 0.611s

<function mytimer.<locals>.wrap at 0x7f8b3eb4aa70>


#### Your answer for Exerciese 0 questions here:

For each factor of ten increase in N, there's approximately a factor of 10 increase in runtime. This slowdown in runtime makes sense because the run time should scale ~linearly with the number of operations it should perform.

<br><br>
### Exercise 1: Naive matrix multiplication: 

For this exercise, we will write our own matrx-matrix multiplication function.  We are goint to be naieve about it, so we want to loop over individual indices, instead of using slicing (in the next section, we will parallelize this code and we want to see how well Numba does at speeding up the naive code).

#### Tasks for this exercise

1. Write a function with that calculates matrix-matrix multiplication such that $C = A\cdot B$, where $A$, $B$, and $C$ are 2D numpy arrays. Make the dtype for the $C$ matrix the same as the dtype for $A$. Below is a stencil you can start with. test your results for accuracy and performance against the np.dot() function. To time the np.dot() function, you can wrap it in another function and use the *mytimer* wrapper; however, please opy the code for testing the A and B dimensions into your function wrapper for the np.dot() function.  This keeps the runtime comparison as a more "apples-to-apples" like comparison.

```python
@mytimer
def my_naive_matmul(A:np.ndarray, B:np.ndarray):
    """ This is a naive matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    # e.g.
    # for i in range(<val>):
    #   for j in range(<val>):
    #     for k in range(<val>):
    #       C[<c_row_index>,<c_col_index>] += A[<a_row_index>,<a_col_index>]*B[<b_row_index>,<b_col_index>]
    
    # 4
    # return the 2D numpy array for C
    return C
```
<br>

2. Define your functions in the cell below.
3. Test and compare the accuracy and runtime of your functions in the next cell.
    - Test with non-square Matrices: $A \in \mathbb{R}^{N\times K}$ and $B \in \mathbb{R}^{K \times M}$. With $N = 64$, $K = 32$, and $M = 128$.
    - Test with a square Matrices: $A,B \in \mathbb{R}^{N\times N}$. For the cases when $N = 64, 128,$ and $256$.
    - Test the following three cases:
        - Case-1. $A$ and $B$ **both** have dtype=np.**float32** (make sure that $C$ is also of dtype=np.**float32**)
        - Case-2. $A$ has dtype=np.**float64**, but $B$ has dtype=np.**float32**
        - Case-3. $A$ and $B$ **both** have dtype=np.**float64** (make sure that $C$ is also of dtype=np.**float64**)
    - For all three case above:
        - Calculate and show the error by computing the sum of the differnece between the $C$ matrices computed from numpy.dot() and your my_naive_matmul() functions. Assume that numpy.dot() is correct
        - Calculate and show the *speedup* that the faster function has versus the slower function.
        - Comment on the which of the three cases is fastest, and comment on what the speedup of the fastest case is and why it is the fastest case.
4. Create your matrices using random numbers. An example is shown below (feel free to copy this).

```python
A = np.random.rand(N, K)
B = np.random.rand("for-you-to-figure-out")
```

<br>

#### Write function definitions for Exercise 1 below

In [5]:
# Define your functions for Exercise 1 here.
@mytimer
def mynaivematmul(A,B):
  global C
  if  A.shape[1] == B.shape[0]:
    Ctype=A.dtype
    C = np.zeros((A.shape[0],B.shape[1]),dtype = Ctype)
    rows=A.shape[0]
    cols=B.shape[1]
    for row in range(rows): 
        for col in range(cols):
            #for elt in range(B.shape[0]):
            for elt in range(A.shape[1]):
              C[row, col] += A[row, elt] * B[elt, col]
    return C
  else:
    return "Can't multiply matrices"

In [6]:
@mytimer
def npcomparison(A,B):
    C=np.dot(A,B)
    return C

<br>

#### Write the test codes for Exercise 1 in the cell below.

In [7]:
# Section Goal: Test your code here for runtime and accuracy
# Task Goal: Test with A=64x32 matrix, B=32x128 matrix

Aa = np.random.rand(64,128)
Ba = np.random.rand(128,32)

print(Ba.shape[1])

Ca = mynaivematmul(Aa,Ba)
Cacomp=npcomparison(Aa,Ba)

print(np.sum(np.abs(Ca-Cacomp)))
print(Ca.shape)

# Task Goal: Test with square matrices: N=64, 128, 256
#N=64
Ab = np.random.rand(64,64)
Bb = np.random.rand(64,64)

print(Bb.shape[1])

Cb = mynaivematmul(Ab,Bb)
Cbcomp=npcomparison(Ab,Bb)

print(Cb.shape)
#N=128

Ac = np.random.rand(128,128)
Bc = np.random.rand(128,128)

print(Bc.shape[1])

Cc = mynaivematmul(Ac,Bc)
Cccomp=npcomparison(Ac,Bc)

print(Cc.shape)
#N=256
Ad = np.random.rand(256,256)
Bd = np.random.rand(256,256)

print(Bd.shape[1])

Cd = mynaivematmul(Ad,Bd)
Cdcomp=npcomparison(Ad,Bd)

print(Cd.shape)

32
Function: 'mynaivematmul({})' executed in 0.147s

Function: 'npcomparison({})' executed in 0.000s

3.637978807091713e-12
(64, 32)
64
Function: 'mynaivematmul({})' executed in 0.148s

Function: 'npcomparison({})' executed in 0.000s

(64, 64)
128
Function: 'mynaivematmul({})' executed in 1.169s

Function: 'npcomparison({})' executed in 0.000s

(128, 128)
256
Function: 'mynaivematmul({})' executed in 9.423s

Function: 'npcomparison({})' executed in 0.001s

(256, 256)


In [8]:
#Task Goal: Testing 3 Cases of different dtypes
A1=np.random.rand(64,64).astype('float32')
B1=np.random.rand(64,64).astype('float32')
C1 = mynaivematmul(A1,B1)
C1comp=npcomparison(A1,B1)
print(np.sum(np.abs(C1-C1comp)))
print(C1.dtype)

A2=np.random.rand(64,64).astype('float64')
B2=np.random.rand(64,64).astype('float32')
C2 = mynaivematmul(A2,B2)
C2comp=npcomparison(A2,B2)
print(np.sum(np.abs(C2-C2comp)))
print(C2.dtype)

A3=np.random.rand(64,64).astype('float64')
B3=np.random.rand(64,64).astype('float64')
C3 = mynaivematmul(A3,B3)
C3comp=npcomparison(A3,B3)
print(np.sum(np.abs(C3-C3comp)))
print(C3.dtype)



Function: 'mynaivematmul({})' executed in 0.146s

Function: 'npcomparison({})' executed in 0.000s

0.0019168854
float32
Function: 'mynaivematmul({})' executed in 0.166s

Function: 'npcomparison({})' executed in 0.000s

3.396394276933279e-12
float64
Function: 'mynaivematmul({})' executed in 0.146s

Function: 'npcomparison({})' executed in 0.000s

3.4798830483850907e-12
float64


<br><br>
Exercise 1 Responses:
- Comment on the which of the three cases is fastest, and comment on what the speedup of the fastest case is and why it is the fastest case.

Of the three cases tested, the fastest case was the case where the A and B matrices are both float32. This is because they are the same dtype and carry less information, thus are able to carry out calculations faster. The slowest case was the case where A and B were different floats. This slowdown is probably due to typecasting, where the B matrix had to be typecast to float64 to match the dtype of the A matrix. Typecasting adds time because it's essentially another operation.

Of the three cases, the most accurate case was the case where the A and B matrices are both float64. This is because they have the highest precision floats of the cases and aren't prone to typecasting errors due to the initial standardization of matrix dtypes. The least accurate case was where the A and B matrices are both float32. This is because they float32 is less precise than its float64 counterpart, meaning this case is more prone to roundoff errors.

<br>


<br><br>
### Exercise 2: Using Numba to speed up matrix multiplication: 

For this exercise, numba to speedup our matrx-matrix multiplication function below. However, there is an interesting twist to this exercise. We will write six versions of the naive function you wrote above. One function for each of the six possible permutations of three loops used to calculate the multplication (i.e. ijk, ikj, jik, etc.)

#### The tasks for this exercise:
1. Make six copies your code for the my_naive_matmul(), one copy for each of the possible permutations and define them in the cell bellow:
    - One of these function will have the same loop order as the my_naive_matmul() function. 
    - However name these functions: numba_mul_\<perm\>(), where \<perm\> should be replaced by the specific loop order of the function.
2. In the cell that follows your function definitions, test and compare the accuracy and runtime of your functions against the numpy.dot() function.
    - Test with a square Matrices: $A,B \in \mathbb{R}^{N\times N}$. For the cases when $N = 128, 256,$ and $512$.
    - For each matrix set their dtype as: dtype=np.float64
    - Calculate and show the error between your functions and the numpy.dot() function. (Same as in Exercise 1.)
    - Calculate and show the *speedup* that the fastest function has versus the all the other functions.
    - You should notice that one of your permutation functions is faster than the others. For this case show the following:
        - Calculate and show the *speedup* that the fastest permutation function has versus the my_naive_matmul().
        - Calculate and show the *speedup* that the fastest permutation function has when $A$, $B$, and $C$ are all of dtype=np.float64 vs all are of dtype=np.float32.
3. Create your matrices using random numbers. (Same as in Exercise 1.) 
4. For each function, you need to add a function decorator for numba. Numba will *JIT* the function (**J**ust **I**n **T**ime compilation). 
    - Use the flag to keep a "cache" of the compiled code (not to be confused with CPU cache). **Note**: to get accurate timings, you will need to run your tests **twice**, because during the first run, the code is compiled and this compile time will be included in your runtime. After the first run, the compiled binary will be stored, so consecutive runs will be faster.
    - Use the flag to use *fast math*
    - Use the flag to disable the Python Global Interpretor Lock (GIL).
    - The code below should get you started, but it is incomplete.
    
```python   
    
@mytimer
@numba.njit(<flagname_caching>=<flag>, <flagname_fast_math>=<flag>, <flagname_no_gil>=<flag>)
def numba_mul_<perm>(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    # e.g.
    # for i in numba.prange(<val>): # !!!! LOOK HERE !!!!
    #   for j in range(<val>):
    #     for k in range(<val>):
    #       C[<c_row_index>,<c_col_index>] += A[<a_row_index>,<a_col_index>]*B[<b_row_index>,<b_col_index>]
    
    # 4
    # return the 2D numpy array for C
    return C
```
    
6. Discuss your results in the markdown cell that follows your codes include in your discussion remarks about the questions asked in the markdown cell.


<br>

#### Write function definitions for Exercise 2 below

In [24]:
# Define your functions for Exercise 2 here.

#original:i,j,k
@mytimer
def numba_mul_ijk(A,B):
  global C
  if  A.shape[1] == B.shape[0]:
    Ctype=A.dtype
    C = np.zeros((A.shape[0],B.shape[1]),dtype = Ctype)
    rows=A.shape[0]
    cols=B.shape[1]
    for row in range(rows): 
        for col in range(cols):
            #for elt in range(B.shape[0]):
            for elt in range(A.shape[1]):
              C[row, col] += A[row, elt] * B[elt, col]
    return C
  else:
    return "Can't multiply matrices"

#variant: i,k,j
@mytimer
def numba_mul_ikj(A,B):
  global C
  if  A.shape[1] == B.shape[0]:
    Ctype=A.dtype
    C = np.zeros((A.shape[0],B.shape[1]),dtype = Ctype)
    rows=A.shape[0]
    cols=B.shape[1]
    for row in range(rows): 
        for elt in range(A.shape[1]):
            for col in range(cols):
                
              C[row, col] += A[row, elt] * B[elt, col]
    return C
  else:
    return "Can't multiply matrices"

#variant: j,i,k
@mytimer
def numba_mul_jik(A,B):
  global C
  if  A.shape[1] == B.shape[0]:
    Ctype=A.dtype
    C = np.zeros((A.shape[0],B.shape[1]),dtype = Ctype)
    rows=A.shape[0]
    cols=B.shape[1]
    for col in range(cols):
        for row in range(rows): 
            #for elt in range(B.shape[0]):
            for elt in range(A.shape[1]):
              C[row, col] += A[row, elt] * B[elt, col]
    return C
  else:
    return "Can't multiply matrices"

#variant: jki
@mytimer
def numba_mul_jki(A,B):
  global C
  if  A.shape[1] == B.shape[0]:
    Ctype=A.dtype
    C = np.zeros((A.shape[0],B.shape[1]),dtype = Ctype)
    rows=A.shape[0]
    cols=B.shape[1]
    for col in range(cols):
        for elt in range(A.shape[1]):
            for row in range(rows): 
              C[row, col] += A[row, elt] * B[elt, col]
    return C
  else:
    return "Can't multiply matrices"

#variant: kij
@mytimer
def numba_mul_kij(A,B):
  global C
  if  A.shape[1] == B.shape[0]:
    Ctype=A.dtype
    C = np.zeros((A.shape[0],B.shape[1]),dtype = Ctype)
    rows=A.shape[0]
    cols=B.shape[1]
    for elt in range(A.shape[1]):
        for row in range(rows): 
            for col in range(cols):
              C[row, col] += A[row, elt] * B[elt, col]
    return C
  else:
    return "Can't multiply matrices"


#variant: kji
@mytimer
def numba_mul_kji(A,B):
  global C
  if  A.shape[1] == B.shape[0]:
    Ctype=A.dtype
    C = np.zeros((A.shape[0],B.shape[1]),dtype = Ctype)
    rows=A.shape[0]
    cols=B.shape[1]
    for elt in range(A.shape[1]):
        for col in range(cols):
            for row in range(rows): 
              C[row, col] += A[row, elt] * B[elt, col]
    return C
  else:
    return "Can't multiply matrices"

In [25]:
@mytimer
def npcomparison(A,B):
    C=np.dot(A,B)
    return C

In [27]:
N=[128,256,512]
i=0
while i<len(N):
    A=np.random.rand(N[i],N[i]).astype('float64')
    B=np.random.rand(N[i],N[i]).astype('float64')
    Ccomp=npcomparison(A,B)
    
    Cijk = numba_mul_ijk(A,B)
    Cikj = numba_mul_ikj(A,B)
    Cjik = numba_mul_jik(A,B)
    Cjki = numba_mul_jki(A,B)
    Ckij = numba_mul_kij(A,B)
    Ckji = numba_mul_kji(A,B)

    print(np.sum(np.abs(Cijk-Ccomp)))
    print(np.sum(np.abs(Cikj-Ccomp)))
    print(np.sum(np.abs(Cjik-Ccomp)))
    print(np.sum(np.abs(Cjki-Ccomp)))
    print(np.sum(np.abs(Ckij-Ccomp)))
    print(np.sum(np.abs(Ckji-Ccomp)))
    
    i+=1

Function: 'npcomparison({})' executed in 0.001s

Function: 'numba_mul_ijk({})' executed in 1.160s

Function: 'numba_mul_ikj({})' executed in 1.158s

Function: 'numba_mul_jik({})' executed in 1.136s

Function: 'numba_mul_jki({})' executed in 1.183s

Function: 'numba_mul_kij({})' executed in 1.170s

Function: 'numba_mul_kji({})' executed in 1.181s

2.7640112421067897e-11
2.7640112421067897e-11
2.7640112421067897e-11
2.7640112421067897e-11
2.7640112421067897e-11
2.7640112421067897e-11
Function: 'npcomparison({})' executed in 0.001s

Function: 'numba_mul_ijk({})' executed in 9.144s

Function: 'numba_mul_ikj({})' executed in 9.295s

Function: 'numba_mul_jik({})' executed in 9.117s

Function: 'numba_mul_jki({})' executed in 9.386s

Function: 'numba_mul_kij({})' executed in 9.301s

Function: 'numba_mul_kji({})' executed in 9.464s

1.4426149164137314e-09
1.4426149164137314e-09
1.4426149164137314e-09
1.4426149164137314e-09
1.4426149164137314e-09
1.4426149164137314e-09
Function: 'npcomparison({}

In [28]:
N=[128,256,512]
i=0
while i<len(N):
    A=np.random.rand(N[i],N[i]).astype('float32')
    B=np.random.rand(N[i],N[i]).astype('float32')
    Ccomp=npcomparison(A,B)
    
    Cijk = numba_mul_ijk(A,B)
    Cikj = numba_mul_ikj(A,B)
    Cjik = numba_mul_jik(A,B)
    Cjki = numba_mul_jki(A,B)
    Ckij = numba_mul_kij(A,B)
    Ckji = numba_mul_kji(A,B)

    print(np.sum(np.abs(Cijk-Ccomp)))
    print(np.sum(np.abs(Cikj-Ccomp)))
    print(np.sum(np.abs(Cjik-Ccomp)))
    print(np.sum(np.abs(Cjki-Ccomp)))
    print(np.sum(np.abs(Ckij-Ccomp)))
    print(np.sum(np.abs(Ckji-Ccomp)))
    
    i+=1

Function: 'npcomparison({})' executed in 0.001s

Function: 'numba_mul_ijk({})' executed in 1.141s

Function: 'numba_mul_ikj({})' executed in 1.161s

Function: 'numba_mul_jik({})' executed in 1.144s

Function: 'numba_mul_jki({})' executed in 1.171s

Function: 'numba_mul_kij({})' executed in 1.166s

Function: 'numba_mul_kji({})' executed in 1.175s

0.015016556
0.015016556
0.015016556
0.015016556
0.015016556
0.015016556
Function: 'npcomparison({})' executed in 0.000s

Function: 'numba_mul_ijk({})' executed in 9.108s

Function: 'numba_mul_ikj({})' executed in 9.307s

Function: 'numba_mul_jik({})' executed in 9.088s

Function: 'numba_mul_jki({})' executed in 9.357s

Function: 'numba_mul_kij({})' executed in 9.368s

Function: 'numba_mul_kji({})' executed in 9.429s

0.11828232
0.11828232
0.11828232
0.11828232
0.11828232
0.11828232
Function: 'npcomparison({})' executed in 0.014s

Function: 'numba_mul_ijk({})' executed in 76.850s

Function: 'numba_mul_ikj({})' executed in 74.741s

Function: 'nu

In [17]:
#Testing numba
'''

@mytimer
@numba.njit(cache=True, parallel=True, fastmath=True, nogil=False)
#@numba.njit(<flagname_caching>=<flag>, <flagname_fast_math>=<flag>, <flagname_no_gil>=<flag>)
#@numba.njit(<flagname_caching>=<flag>, <flagname_fast_math>=<flag>, <flagname_no_gil>=<flag>)
def numba_mul_ijk(A,B):
  global C
  if  A.shape[1] == B.shape[0]:
    Ctype=A.dtype
    C = np.zeros((A.shape[0],B.shape[1]),dtype = Ctype)
    rows=A.shape[0]
    cols=B.shape[1]
    for row in numba.prange(rows): 
        for col in numba.prange(cols):
            #for elt in range(B.shape[0]):
            for elt in numba.prange(A.shape[1]):
              C[row, col] += A[row, elt] * B[elt, col]
    return C
  else:
    return "Can't multiply matrices"
    
    #Didn't work because njit has trouble reading Python. Also, be careful for the if statements. Parallelization 
    #makes multiple branches. This means that the program gets confused on waiting and chooseing what to execute.
    
'''

@mytimer
@numba.njit(cache=True, parallel=True, fastmath=True, nogil=True)
def numba_mul_ijk(A,B):
    
    if A.shape[1] != B.shape[0]:
        raise Exception('A and B have incompatible dims')
        
    C = np.zeros((A.shape[0],B.shape[1]),dtype=A.dtype)
    rows=A.shape[0]
    cols=B.shape[1]
    
    for row in numba.prange(rows): 
        for col in range(cols):
            for elt in range(A.shape[1]):
                C[row, col] += A[row, elt] * B[elt, col]
    return C


In [24]:
@mytimer
@numba.njit(cache=True, parallel=True, fastmath=True, nogil=True)
def numba_mul_ikj(A,B):
    
    if A.shape[1] != B.shape[0]:
        raise Exception('A and B have incompatible dims')
        
    C = np.zeros((A.shape[0],B.shape[1]),dtype=A.dtype)
    rows=A.shape[0]
    cols=B.shape[1]
    
    for row in numba.prange(rows): 
        for elt in range(A.shape[1]):
            for col in range(cols):
                C[row, col] += A[row, elt] * B[elt, col]
    return C

@mytimer
@numba.njit(cache=True, parallel=True, fastmath=True, nogil=True)
def numba_mul_jik(A,B):
    
    if A.shape[1] != B.shape[0]:
        raise Exception('A and B have incompatible dims')
        
    C = np.zeros((A.shape[0],B.shape[1]),dtype=A.dtype)
    rows=A.shape[0]
    cols=B.shape[1]
    
    for col in numba.prange(cols):
        for row in range(rows): 
            for elt in range(A.shape[1]):
                C[row, col] += A[row, elt] * B[elt, col]
    return C

@mytimer
@numba.njit(cache=True, parallel=True, fastmath=True, nogil=True)
def numba_mul_jki(A,B):
    
    if A.shape[1] != B.shape[0]:
        raise Exception('A and B have incompatible dims')
        
    C = np.zeros((A.shape[0],B.shape[1]),dtype=A.dtype)
    rows=A.shape[0]
    cols=B.shape[1]
    
    for col in numba.prange(cols):
        for elt in range(A.shape[1]):
            for row in range(rows): 
                C[row, col] += A[row, elt] * B[elt, col]
    return C

@mytimer
@numba.njit(cache=True, parallel=True, fastmath=True, nogil=True)
def numba_mul_kij(A,B):
    
    if A.shape[1] != B.shape[0]:
        raise Exception('A and B have incompatible dims')
        
    C = np.zeros((A.shape[0],B.shape[1]),dtype=A.dtype)
    rows=A.shape[0]
    cols=B.shape[1]
    
    for elt in numba.prange(A.shape[1]):
        for row in range(rows): 
            for col in range(cols):
                C[row, col] += A[row, elt] * B[elt, col]
    return C

@mytimer
@numba.njit(cache=True, parallel=True, fastmath=True, nogil=True)
def numba_mul_kji(A,B):
    
    if A.shape[1] != B.shape[0]:
        raise Exception('A and B have incompatible dims')
        
    C = np.zeros((A.shape[0],B.shape[1]),dtype=A.dtype)
    rows=A.shape[0]
    cols=B.shape[1]
    
    for elt in numba.prange(A.shape[1]):
        for col in range(cols):
            for row in range(rows): 
                C[row, col] += A[row, elt] * B[elt, col]
    return C

In [25]:
N=[128,256]
i=0
while i<len(N):
    A=np.random.rand(N[i],N[i]).astype('float32')
    B=np.random.rand(N[i],N[i]).astype('float32')
    Ccomp=npcomparison(A,B)
    
    Cijk = numba_mul_ijk(A,B)
    Cikj = numba_mul_ikj(A,B)
    Cjik = numba_mul_jik(A,B)
    Cjki = numba_mul_jki(A,B)
    Ckij = numba_mul_kij(A,B)
    Ckji = numba_mul_kji(A,B)
    
    print(np.sum(np.abs(Cijk-Ccomp)))
    print(np.sum(np.abs(Cikj-Ccomp)))
    print(np.sum(np.abs(Cjik-Ccomp)))
    print(np.sum(np.abs(Cjki-Ccomp)))


    i+=1

Function: 'npcomparison({})' executed in 0.008s

Function: 'numba_mul_ijk({})' executed in 0.014s

Function: 'numba_mul_ikj({})' executed in 1.483s

Function: 'numba_mul_jik({})' executed in 1.383s

Function: 'numba_mul_jki({})' executed in 1.322s

Function: 'numba_mul_kij({})' executed in 1.371s

Function: 'numba_mul_kji({})' executed in 1.451s

0.0
0.0
0.0
0.0
1632.0574
14362.732
Function: 'npcomparison({})' executed in 0.008s

Function: 'numba_mul_ijk({})' executed in 0.024s

Function: 'numba_mul_ikj({})' executed in 0.030s

Function: 'numba_mul_jik({})' executed in 0.026s

Function: 'numba_mul_jki({})' executed in 0.040s

Function: 'numba_mul_kij({})' executed in 0.004s

Function: 'numba_mul_kji({})' executed in 0.042s

0.0
0.0
0.0
0.0
608.27234
31695.443


<br>

#### Write the test codes for Exercise 2 in the cell below.

In [ ]:
# Test your code here for runtimes, accuracy, and speedups


<br>

### Discussion for Exercise 2
1. What do you think is causing the differences in performance between the various permutations?
1. Which function is fastest (permutations or numpy.dot)? Why do you think this function is the fastest, and could there be multiple factors involved regarding the superior performance?
1. When comparing the matrix-matrix performance between the cases were all matrices had dtype=np.float64 vs.  all matrices having dtype=np.float32, which dtype was fastest? Roughtly what was the speedup when using this dtype vs the other?
1. Does Amdahl's Law play a major factor in the performance differences? Which part of the matrix-matrix multiplication was not parallelized (serial portion)? (Hint: which matricies did we reuse for each function?) Could we parallelize this part, and if so, are there caveats?
1. Do you think all codes should be parallelized? What about matrix-matrix multiplication? (This is a subjective question, but I am looking for a brief, but rational and informed arguement).
1. For those taking the class for **4 units**: Regarding the performance differences, which Law do you think is more relevant when comparing the performance differences between each of the functions in this exercise, Amdahl's Law or Gustafson's Law? Explain your reasoning. 

#1 I think that the differences is caused by cache efficiency. It is best to take the direction where the fastest caches are being utilized. This direction might be dependent on the program that you use (ex. Python vs C).

#2 The function that performed the fastest was the numpy.dot operation. This is because it runs off of already efficient C code. The parralelization-compatible wraparounds caused by the permutations will always have slightly more operations than this numpy.dot function, meaning the run time must always be longer for the permutaions.

#3 Operations with the float32 dtype were approximately twice as fast as their float64 counterparts. The reason for this is similar to the explanation described in Exercise 1.

#4 Amdahl's Law states "the overall performance improvement gained by optimizing a single part of a system is limited by the fraction of time that the improved part is actually used". I think that this law plays an important factor in the performance differences (parallel vs not), because it increased the efficiency of the algebraic operations involved in the matrix-matrix multiplication process. By parallelizing the algebraic processes, the program divides the workload of the slowest part of the code, increasing the overall speed. In the code, the matrix vectors (1D row or column) are not parallelized. It would be possible to parallelize this part; however, this would take a lot of effort and time. Not worth yo.

#5 I don't think that all codes should be parallelized. It takes time for the original code to be rewritten ST it is compatible with Numba. This might be more of a slowdown for the programmer, especially if the code isn't computationally expensive. I do however believe that most matrix-matrix multiplication codes should be parallelized. This is because you get a dramatic reduction in run-time. Chances are that if you need to do matrix multiplications, you'll to perform various operations. This is more than enough just cause to warrant parallelizaiton.
